In [1]:
import os
import sys
sys.path.append('../code')
import importlib

import numpy as np  
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr

import SeaLevelContrib as slc

In [10]:
importlib.reload(slc)

<module 'SeaLevelContrib' from '../code/SeaLevelContrib.py'>

In [12]:
tg_id=[20, 22, 23, 24, 25, 32]
gia_df = pd.DataFrame()
for idx, tgi in enumerate(tg_id):
    if idx == 0:
        gia_df = slc.GIA_ICE6G(tgi)
        gia_df = gia_df.rename(columns = {'GIA' : tgi})
    else:
        gia_df[tgi] = slc.GIA_ICE6G(tgi)
gia_df['mean'] = gia_df.mean(axis=1)
gia_df

,20,22,23,24,25,32,mean
time,,,,,,,
1900,0.000,0.000,0.00,0.000,0.000,0.000,0.000000
1901,0.004,0.017,0.05,0.061,0.058,0.031,0.036833
1902,0.008,0.034,0.10,0.122,0.116,0.062,0.073667
1903,0.012,0.051,0.15,0.183,0.174,0.093,0.110500
1904,0.016,0.068,0.20,0.244,0.232,0.124,0.147333
...,...,...,...,...,...,...,...
2015,0.460,1.955,5.75,7.015,6.670,3.565,4.235833
2016,0.464,1.972,5.80,7.076,6.728,3.596,4.272667
2017,0.468,1.989,5.85,7.137,6.786,3.627,4.309500


In [17]:
# GIA in mm/year
(gia_df.loc[2019] - gia_df.loc[2018])*10
# Where does the 1.1 come from in Vermersen?

20      0.040000
22      0.170000
23      0.500000
24      0.610000
25      0.580000
32      0.310000
mean    0.368333
dtype: float64

In [11]:
lev =  slc.LevitusSL(extrap=True, extrap_back=True)
lev

,GloSteric
time,
1950,-1.480427
1951,-1.442732
1952,-1.405037
1953,-1.367342
1954,-1.329647
...,...
2016,2.662499
2017,2.710927
2018,2.784583


In [7]:
slc.contrib_frederikse2020_glob('Steric')

,GloSteric
time,
1900,-4.827935
1901,-4.914490
1902,-4.987756
1903,-5.110059
1904,-5.224018
...,...
2014,0.467003
2015,0.674470
2016,0.647246


In [13]:
import netCDF4
def make_waqua_df(tg_id):
    '''Read time series of annually averaged sea level from the WAQUA model forced by ERA-interim.'''
    dir_waqua = '../data/DataWAQUANinaERAI'
    ds_wa = netCDF4.Dataset(dir_waqua+'/ERAintWAQUA_waterlevels_speed_1979_2015.nc')

    # Get WAQUA tide gauge names that are not editted in the same way as PSMSL names
    tg_data_dir = '../data/rlr_annual'
    names_col = ('id', 'lat', 'lon', 'name', 'coastline_code', 'station_code', 'quality')
    filelist_df = pd.read_csv(tg_data_dir + '/filelist.txt', sep=';', header=None, names=names_col)
    filelist_df = filelist_df.set_index('id')
    tgn = filelist_df.name[tg_id].replace(' ', '').lower()[:8]
    
    dh = ds_wa[tgn + '/WAQUA_surge'][:]*100
    time_wa = ds_wa['time'][:]
    t_wa = netCDF4.num2date(time_wa, ds_wa.variables['time'].units)

    t_wa_y = np.empty_like(t_wa)
    
    for i in range(len(t_wa)):
        t_wa_y[i] = t_wa[i].year
        
    waqua_df = pd.DataFrame( data = dict( time=t_wa_y, WindPressure=dh.data) )
    waqua_df = waqua_df.set_index('time')
    
    return waqua_df
make_waqua_df(89)

KeyError: 'hirtshal'

In [19]:
tg_id = 754

'''Read time series of annually averaged sea level from the WAQUA model forced by ERA-interim.'''
dir_waqua = '../data/DataWAQUANinaERAI'
ds_wa = netCDF4.Dataset(dir_waqua+'/ERAintWAQUA_waterlevels_speed_1979_2015.nc')

# Get WAQUA tide gauge names that are not editted in the same way as PSMSL names
tg_data_dir = '../data/rlr_annual'
names_col = ('id', 'lat', 'lon', 'name', 'coastline_code', 'station_code', 'quality')
filelist_df = pd.read_csv(tg_data_dir + '/filelist.txt', sep=';', header=None, names=names_col)
filelist_df = filelist_df.set_index('id')
print(filelist_df.name[tg_id])
tgn = filelist_df.name[tg_id].replace('-', '').replace(' ', '')
tgn = tgn.lower()[:8]

dh = ds_wa[tgn + '/WAQUA_surge'][:]*100

 LOWESTOFT                               


In [11]:
tgn.replace('-','')

'westter'

In [10]:
ds_wa

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    description: File containing WAQUA surge model output forced with instantaneous wind stress from ERA Interim reanalysis dataset.
    history: Wed Jun 12 14:55:45 2019: ncrcat ERAintWAQUA_waterlevels_speed_1979_avg.nc ERAintWAQUA_waterlevels_speed_1980_avg.nc ERAintWAQUA_waterlevels_speed_1981_avg.nc ERAintWAQUA_waterlevels_speed_1982_avg.nc ERAintWAQUA_waterlevels_speed_1983_avg.nc ERAintWAQUA_waterlevels_speed_1984_avg.nc ERAintWAQUA_waterlevels_speed_1985_avg.nc ERAintWAQUA_waterlevels_speed_1986_avg.nc ERAintWAQUA_waterlevels_speed_1987_avg.nc ERAintWAQUA_waterlevels_speed_1988_avg.nc ERAintWAQUA_waterlevels_speed_1989_avg.nc ERAintWAQUA_waterlevels_speed_1990_avg.nc ERAintWAQUA_waterlevels_speed_1991_avg.nc ERAintWAQUA_waterlevels_speed_1992_avg.nc ERAintWAQUA_waterlevels_speed_1993_avg.nc ERAintWAQUA_waterlevels_speed_1994_avg.nc ERAintWAQUA_waterlevels_speed_1995_avg.nc ERAintWAQUA_waterleve